In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import os

# Veri seti ve NLTK gereksinimlerini yükleme
nltk.download('stopwords')
nltk.download('wordnet')

# Modeli kaydetme ve yükleme için dosya yolları
model_filename = '../pythonProject/trained_model.pkl'
vectorizer_filename = '../pythonProject/tfidf_vectorizer.pkl'

def preprocess_text(text):
    text = text.lower()  # Küçük harfe çevir
    text = text.translate(str.maketrans('', '', string.punctuation))  # Noktalama işaretlerini kaldır
    return text

lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

def train_and_save_model():
    print("Veri yükleniyor...")

    # Dataset'i yükleme
    file_path = 'ml_dataset.csv'
    df = pd.read_csv(file_path)

    print("Veri yükleme tamamlandı.")
    print("İlk birkaç satır:")
    print(df.head())

    # Sütun adlarını yazdırma
    print(df.columns)

    # Gerekli sütunları seçme ve eksik verileri temizleme
    # Doğru sütun adlarını kullanın
    df = df[['Country', 'Ingredients']].dropna()

    print("Veri temizleme tamamlandı.")

    # Veri ön işleme
    df['Ingredients'] = df['Ingredients'].apply(preprocess_text)
    df['Ingredients'] = df['Ingredients'].apply(lemmatize_text)

    print("Veri ön işleme tamamlandı.")

    # TF-IDF vektörizer ve Logistic Regression modeli pipeline oluşturma
    vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), ngram_range=(1, 2))
    X = vectorizer.fit_transform(df['Ingredients'])
    y = df['Country']

    print("Veri seti eğitim ve test setlerine ayrılıyor...")
    # Veri setini eğitim ve test setlerine ayırma
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Model eğitiliyor...")
    # Modeli eğitme
    model = LogisticRegression(max_iter=1000, C=1.0)
    model.fit(X_train, y_train)

    print("Model eğitildi. Test seti ile doğruluk ölçülüyor...")
    # Test seti ile modelin doğruluğunu ölçme
    y_pred = model.predict(X_test)
    print(f'Test set accuracy: {accuracy_score(y_test, y_pred)}')

    print("Model ve vektörizer kaydediliyor...")
    # Modeli ve vektörizeri kaydetme
    joblib.dump(model, model_filename)
    joblib.dump(vectorizer, vectorizer_filename)
    print("Model ve vektörizer başarıyla kaydedildi.")

def load_model():
    if os.path.exists(model_filename) and os.path.exists(vectorizer_filename):
        print("Model ve vektörizer yükleniyor...")
        model = joblib.load(model_filename)
        vectorizer = joblib.load(vectorizer_filename)
        print("Model ve vektörizer başarıyla yüklendi.")
        return model, vectorizer
    else:
        print("Kaydedilmiş model bulunamadı. Model eğitiliyor...")
        train_and_save_model()
        return load_model()

def predict_country(ingredients):
    model, vectorizer = load_model()
    ingredients = preprocess_text(ingredients)
    ingredients = lemmatize_text(ingredients)
    ingredients_vector = vectorizer.transform([ingredients])
    country_prediction = model.predict(ingredients_vector)
    return country_prediction[0]

# Kullanıcıdan malzemeleri alma ve tahmin yapma
query = input("Enter at least 5 ingredient in comma-seperated form")
liste = query.split((","))
if len(liste)>=5:
    user_ingredients = "tomato, garlic, onion, olive oil"
    predicted_country = predict_country(user_ingredients)
    print(f'The predicted country for the given ingredients is: {predicted_country}')
else:
    print("Invalid input")

